In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support, confusion_matrix
from nltk.stem.snowball import SnowballStemmer
from sklearn.grid_search import GridSearchCV
from sklearn.svm import SVC
from sklearn import datasets
from nltk.tokenize import RegexpTokenizer
from gensim.sklearn_api.w2vmodel import W2VTransformer
from nltk.corpus import stopwords

c:\users\m6800\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\m6800\appdata\local\programs\python\python36\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
def row(key, scores, params):
            d = {
                 'estimator': key,
                 'mean_score': np.mean(scores),
                 'f1-scores': scores,
            }
            return pd.Series({**params,**d})
tokenizer = RegexpTokenizer(r'\w+')

In [3]:
%%time
df = pd.read_csv("Handelingen.csv", index_col=0)
Partij = df.partij
sbs = SnowballStemmer("dutch")
Tekst = df.tekst.apply(lambda x: ' '.join([sbs.stem(t) for t in tokenizer.tokenize(x)]))
del df

Wall time: 4min 44s


In [10]:
df = pd.read_csv('Models.tsv', sep ='\t', index_col=0)
df3 = df.loc[df.DOEN == True]
models = {}
for x,y in zip(df3.Classifier,df3.PIPELINE):
    exec(compile("a="+y,'','exec'))
    models[x] = Pipeline(a)    
params = {}
for clf in set(df3.Classifier):
    test = df3.loc[df3.Classifier == clf]
    test = test.loc[test.DOEN == True]
    test = test.dropna(axis=1, how='all')
    test= test.drop([col for col in test.columns if "__" not in col], axis =1)
    test = test.to_dict(orient='list')
    testdict = {}
    for x,y in test.items():
        exec(compile("a="+y[0],'','exec'))
        testdict[x] = a   
    params[clf] = testdict
#rawdict = {x:y for x,y in zip(df3.Classifier,df3.RAW)}
df

,Unnamed: 0,DOEN,RAW,Classifier,PIPELINE,SGD__loss,MNB__alpha,KNB__n_neighbors,SGD__penalty,MLP__hidden_layer_sizes,MLP__solver,MLP__alpha,SVC__degree,vect__ngram_range,vect__min_df
2,True,False,SVM,"[('vect',CountVectorizer(stop_words=stopwords....",NaN,NaN,NaN,"['none', 'l2', 'l1', 'elasticnet']",NaN,NaN,NaN,NaN,NaN,"[(1,1),(1,2)]","[5,10,50,100]"
0,True,False,LogisticClassifier,"[('vect',CountVectorizer(stop_words=stopwords....",NaN,NaN,NaN,"['none', 'l2', 'l1', 'elasticnet']",NaN,NaN,NaN,NaN,"[(1,1),(1,2)]","[5,10,50,100]",NaN
1,True,False,MultinomialNB,"[('vect',CountVectorizer(stop_words=stopwords....",NaN,"[0,0.5,1]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,False,SVC,"[('vect',CountVectorizer(stop_words=stopwords....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[2,3]",NaN,NaN,NaN
5,True,False,MLP,"[('vect',CountVectorizer(stop_words=stopwords....",NaN,NaN,NaN,NaN,NaN,NaN,"[0.0001,0.0002]",NaN,NaN,NaN,NaN


In [8]:
for name,pipe in models.items():
    gs = GridSearchCV(pipe, params[name], cv=5, scoring='f1_weighted')
    gs.fit(Tekst, Partij)
    rows = [row(name, gsc.cv_validation_scores, gsc.parameters) for gsc in gs.grid_scores_]
    df2 = pd.concat(rows, axis=1).T
    pd.concat([df2,pd.read_csv('Scores.csv', index_col=0)], ignore_index=True).sort_values(['mean_score'], ascending=False).to_csv('Scores.csv')
    df.loc[df.Classifier==name,"DOEN"]=False
    df.to_csv('Models.tsv', sep='\t')
    
#https://stackoverflow.com/questions/46735847/save-best-params-in-gridsearch-in-a-pandas-dataframe
# https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value?rq=1

In [6]:
3	True	False	W2V	[('w2v', W2VTransformer()), ('SGD', SGDClassifier(loss='hinge'))]				['none', 'l2', 'l1', 'elasticnet']						


SyntaxError: invalid syntax (<ipython-input-6-9541d2eedbae>, line 1)